# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [8]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [9]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [10]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [37]:
song_files = get_files('data/song_data/')

In [38]:
filepath = song_files[0]

In [39]:
df = pd.read_json(filepath, lines=True, dtype={'year': int})
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNNKDK1187B98BBD5,45.80726,Zagreb Croatia,15.9676,Jinx,407.37914,1,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),0


In [14]:
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Series([], dtype: float64)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
artist_id           1 non-null object
artist_latitude     1 non-null float64
artist_location     1 non-null object
artist_longitude    1 non-null float64
artist_name         1 non-null object
duration            1 non-null float64
num_songs           1 non-null int64
song_id             1 non-null object
title               1 non-null object
year                1 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 160.0+ bytes


In [16]:
df.loc[:, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]

['SOFNOQK12AB01840FC',
 'Kutt Free (DJ Volume Remix)',
 'ARNNKDK1187B98BBD5',
 0,
 407.37914]

In [17]:
df.loc[0, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()

['SOFNOQK12AB01840FC',
 'Kutt Free (DJ Volume Remix)',
 'ARNNKDK1187B98BBD5',
 0,
 407.37914000000001]

In [18]:
song_data = df.loc[0, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()
song_data

['SOFNOQK12AB01840FC',
 'Kutt Free (DJ Volume Remix)',
 'ARNNKDK1187B98BBD5',
 0,
 407.37914000000001]

In [19]:
song_data[3] = song_data[3].item()
song_data[4] = song_data[4].item()

In [20]:
type(song_data[3])

int

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [21]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [22]:
artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

In [23]:
artist_data = df.loc[0, artist_columns].values.tolist()
artist_data

['ARNNKDK1187B98BBD5',
 'Jinx',
 'Zagreb Croatia',
 45.807259999999999,
 15.967600000000001]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [24]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [25]:
log_files = get_files('data/log_data/')

In [26]:
filepath = log_files[0]

In [27]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 18 columns):
artist           319 non-null object
auth             366 non-null object
firstName        352 non-null object
gender           352 non-null object
itemInSession    366 non-null int64
lastName         352 non-null object
length           319 non-null float64
level            366 non-null object
location         352 non-null object
method           366 non-null object
page             366 non-null object
registration     352 non-null float64
sessionId        366 non-null int64
song             319 non-null object
status           366 non-null int64
ts               366 non-null int64
userAgent        352 non-null object
userId           366 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 51.5+ KB


In [29]:
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

artist          47
firstName       14
gender          14
lastName        14
length          47
location        14
registration    14
song            47
userAgent       14
dtype: int64

In [32]:
df[null_columns].loc[342:362]

,artist,firstName,gender,lastName,length,location,registration,song,userAgent
342,Alliance Ethnik,Tegan,F,Levine,195.94404,"Portland-South Portland, ME",1.540794e+12,SinceritÃÂ© Et Jalousie,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
343,Muse,Tegan,F,Levine,365.45261,"Portland-South Portland, ME",1.540794e+12,New Born,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
344,Steps,Tegan,F,Levine,201.76934,"Portland-South Portland, ME",1.540794e+12,5_ 6_ 7_ 8,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
345,John Mayer,Tegan,F,Levine,198.19057,"Portland-South Portland, ME",1.540794e+12,The Heart Of Life,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
346,Kings Of Leon,Sean,F,Wilson,201.79546,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",1.540370e+12,Revelry,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
347,David Gray,Tegan,F,Levine,263.33995,"Portland-South Portland, ME",1.540794e+12,Draw The Line,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
348,King Stitt,Sean,F,Wilson,176.27383,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",1.540370e+12,The Ugly One (AKA Lee Van Cleef),"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
349,Tub Ring,Tegan,F,Levine,233.69098,"Portland-South Portland, ME",1.540794e+12,Invalid,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
350,Era,Tegan,F,Levine,200.56771,"Portland-South Portland, ME",1.540794e+12,Cathar Rhythm,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
351,None,Tegan,F,Levine,NaN,"Portland-South Portland, ME",1.540794e+12,None,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df.size

6588

In [20]:
df = df[df['page'] == 'NextSong']
df.size

5742

In [21]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [22]:
test = df.copy()

In [23]:
import datetime

In [24]:
test['ts'] = test['ts'].apply(lambda ts: datetime.datetime.fromtimestamp(ts/1000.0))
test.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,2018-11-29 00:00:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,2018-11-29 00:01:30.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,2018-11-29 00:04:01.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,2018-11-29 00:04:55.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,2018-11-29 00:07:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [25]:
t = df.copy()
t.ts = t['ts'].apply(lambda ts: datetime.datetime.fromtimestamp(ts/1000.0))
t.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,2018-11-29 00:00:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,2018-11-29 00:01:30.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,2018-11-29 00:04:01.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,2018-11-29 00:04:55.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,2018-11-29 00:07:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [26]:
t['ts'].dt.weekday_name.values

array(['Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursday',
       'Thursday', 'Thursday', 'Thursday', 'Thursday', 'Thursd

In [27]:
time_data = (t['ts'], t['ts'].dt.hour, t['ts'].dt.day, t['ts'].dt.week, t['ts'].dt.month, t['ts'].dt.year, t['ts'].dt.weekday_name)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [28]:
import numpy as np

In [29]:
time_dict = dict(zip(column_labels, time_data))
len(time_dict)

7

In [30]:
time_df = pd.DataFrame(time_dict)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-29 00:00:57.796,0,29,48,11,2018,Thursday
1,2018-11-29 00:01:30.796,0,29,48,11,2018,Thursday
2,2018-11-29 00:04:01.796,0,29,48,11,2018,Thursday
3,2018-11-29 00:04:55.796,0,29,48,11,2018,Thursday
4,2018-11-29 00:07:13.796,0,29,48,11,2018,Thursday


In [31]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 0 to 365
Data columns (total 7 columns):
start_time    319 non-null datetime64[ns]
hour          319 non-null int64
day           319 non-null int64
week          319 non-null int64
month         319 non-null int64
year          319 non-null int64
weekday       319 non-null object
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 19.9+ KB


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [32]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [33]:
user_columns = ['userId', 'firstName', 'lastName', 'gender', 'level']
user_df = df.loc[:, user_columns]
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 0 to 365
Data columns (total 5 columns):
userId       319 non-null object
firstName    319 non-null object
lastName     319 non-null object
gender       319 non-null object
level        319 non-null object
dtypes: object(5)
memory usage: 15.0+ KB


In [34]:
user_df = user_df.drop_duplicates(subset='userId')
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 365
Data columns (total 5 columns):
userId       27 non-null object
firstName    27 non-null object
lastName     27 non-null object
gender       27 non-null object
level        27 non-null object
dtypes: object(5)
memory usage: 1.3+ KB


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [35]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [36]:
df.ts = df['ts'].apply(lambda ts: datetime.datetime.fromtimestamp(ts/1000.0))
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,2018-11-29 00:00:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,2018-11-29 00:01:30.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,2018-11-29 00:04:01.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,2018-11-29 00:04:55.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,2018-11-29 00:07:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [37]:
print(songplay_table_insert)


    INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)



In [38]:
print(songplay_table_create)


    CREATE TABLE IF NOT EXISTS songplays (songplay_id SERIAL PRIMARY KEY, start_time timestamp, user_id int, level varchar, song_id varchar, artist_id varchar, session_id int, location varchar, user_agent varchar);



In [39]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [40]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.